# Styles

The REST API allows you to list, create, upload, update, and delete styles in GeoServer.

> **Note:** <br> 
> You can find the official example at [https://docs.geoserver.org/2.25.x/en/user/rest/styles.html](https://docs.geoserver.org/2.25.x/en/user/rest/styles.html)

## Setup

### Imports

First, we need to import the necessary modules and classes.

In [1]:
from pathlib import Path
from geoserver import GeoServer

### GeoServer Connection

Connect to the running GeoServer instance and create a workspace and a store.

In [2]:
# Setup the geoserver instance
geoserver = GeoServer(
    service_url="http://localhost:8080/geoserver",
    username="admin",
    password="geoserver",
)

Let's cleanup the previous styles:

In [3]:
if geoserver.workspace_exists("demo"):
    geoserver.delete_workspace("demo", recurse=True)

geoserver.create_workspace_from_name("demo")

'Created'

### Utils

For this tutorial, styles are located in the [examples](https://github.com/arthurdjn/geoserver-py/tree/main/examples) directory.

In [4]:
# Directory containing sample data
DATA_DIR = Path("../tests/data")
assert DATA_DIR.exists(), f"The directory {DATA_DIR} does not exist."

## Listing all styles

To list all the styles in the GeoServer instance, use the `get_styles` method.

In [5]:
geoserver.get_styles()

{'styles': {'style': [{'name': 'generic',
    'href': 'http://localhost:8080/geoserver/rest/styles/generic.json'},
   {'name': 'line',
    'href': 'http://localhost:8080/geoserver/rest/styles/line.json'},
   {'name': 'point',
    'href': 'http://localhost:8080/geoserver/rest/styles/point.json'},
   {'name': 'polygon',
    'href': 'http://localhost:8080/geoserver/rest/styles/polygon.json'},
   {'name': 'raster',
    'href': 'http://localhost:8080/geoserver/rest/styles/raster.json'}]}}

## Retrieve a style

To get a specific style, use the `get_style` method.

In [6]:
geoserver.get_style(style="point")

{'style': {'name': 'point',
  'format': 'sld',
  'languageVersion': {'version': '1.0.0'},
  'filename': 'default_point.sld'}}

## Creating a style

You can create a new style on the server in two ways. In the first way, the creation is done in two steps: the style entry is created in the catalog, and then the style content is uploaded.
The second way can add the style to the server in a single step by uploading a file containing the style content.


### Create a new style in two steps

In [7]:
# Using XML format
body = """
<style>
    <name>elevation</name>
    <filename>elevation.sld</filename>
</style>
"""

geoserver.create_style(body=body, workspace="demo")

'Created'

In [8]:
# Get the styles associated to the raster layer
file_path = DATA_DIR / "styles" / "elevation.sld"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


with open(file_path, "r") as file:
    body = file.read()


geoserver.update_style(style="elevation", body=body, workspace="demo")

'Updated'

### Uploading a style from a file

You can upload a style from a file using the `upload_style` method. The method supports both SLD and ZIP files.

In [9]:
# Get the styles associated to the raster layer
file_path = DATA_DIR / "styles" / "elevation.sld"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


geoserver.upload_style(file=file_path, workspace="demo", style="elevation", overwrite=True)

'Created'

## Changing an existing style

To update a style, use the `update_style` method.

In [10]:
# Get the styles associated to the raster layer
file_path = DATA_DIR / "styles" / "elevation.sld"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


with open(file_path, "r") as file:
    body = file.read()


geoserver.update_style(style="elevation", body=body, workspace="demo")

'Updated'

## Reupload the content of an existing style

By default, you cannot reupload the content of an existing style. You can use the `overwrite=True` parameter to allow the content to be reuploaded.

In [11]:
geoserver.upload_style(file=file_path, workspace="demo", style="elevation", overwrite=True)

'Created'

## Downloading a style

The SLD itself can be downloaded using the `download_style` method.

In [12]:
xml = geoserver.download_style(style="elevation", workspace="demo")
print(xml)

<?xml version="1.0" encoding="UTF-8"?><sld:StyledLayerDescriptor xmlns="http://www.opengis.net/sld" xmlns:sld="http://www.opengis.net/sld" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" version="1.0.0">
  <sld:NamedLayer>
    <sld:Name>raster</sld:Name>
    <sld:UserStyle>
      <sld:Name>raster</sld:Name>
      <sld:FeatureTypeStyle>
        <sld:Name>name</sld:Name>
        <sld:Rule>
          <sld:RasterSymbolizer>
            <sld:ChannelSelection>
              <sld:GrayChannel>
                <sld:SourceChannelName>1</sld:SourceChannelName>
                <sld:ContrastEnhancement>
                  <sld:GammaValue>1.0</sld:GammaValue>
                </sld:ContrastEnhancement>
              </sld:GrayChannel>
            </sld:ChannelSelection>
            <sld:ColorMap>
              <sld:ColorMapEntry color="#FFFFFF" opacity="0" quantity="-1" label="label"/>
              <sld:ColorMapEntry color="#0000FF" opacity="1" quantity="0" label="label"

## Publishing a style

To style an existing layer, use the `publish_style` method. This method is just a shortcut to the `update_layer` method, which allows you to update the style of a layer.

In [13]:
file_path = DATA_DIR / "rasters" / "elevation.tif"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


geoserver.upload_coverage_store(file=file_path, workspace="demo", format="geotiff")

'Created'

In [14]:
geoserver.publish_style(layer="elevation", style="elevation", workspace="demo")

'Updated'

## Deleting a style

To delete a style, use the `delete_style` method. You can use the `purge=True` argument to remove the style and all its references.

In [15]:
geoserver.get_styles(workspace="demo")

{'styles': {'style': [{'name': 'elevation',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/styles/elevation.json'}]}}

In [16]:
geoserver.delete_style(style="elevation", workspace="demo", recurse=True)

'Deleted'